In [6]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "CS340zb!"

# Connect to database via CRUD Module
db = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)
    
# Ensure numeric coordinates if present
for col in ('location_lat', 'location_long'):
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # exact filename
if not os.path.exists(image_filename):
    for alt in ['code_files/Grazioso Salvare Logo.png', 'codio_files/code_files/Grazioso Salvare Logo.png']:
        if os.path.exists(alt):
            image_filename = alt
            break
encoded_image = None
try:
    with open(image_filename, 'rb') as f:
        encoded_image = base64.b64encode(f.read()).decode()
except Exception:
    pass  # If not found, we simply won't render the image

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Zach Bakke - CS-340 7-2 Project Two Dashboard'))),    
    html.Center(
        html.Img(
            src=f"data:image/png;base64,{encoded_image}",
            alt="Grazioso Salvare logo",
            style={"height": "80px"}
        )
    )  if encoded_image else html.Div(),
    html.Hr(),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([     
        html.H4("Rescue Type Filter"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All (no filter)', 'value': 'ALL'},
                {'label': 'Water Rescue', 'value': 'WATER'},
                {'label': 'Mountain/Wilderness', 'value': 'MOUNTAIN'},
                {'label': 'Disaster/Tracking', 'value': 'DISASTER'}
            ],
            value='ALL',
            inline=True
        ),

        html.Br(),

        html.Div(
            style={'display': 'flex', 'gap': '24px', 'flexWrap': 'wrap'}, children=[
                html.Div(style={'minWidth': '280px'}, children=[
                    html.Label("Breed"),
                    dcc.Dropdown(
                        id='breed-dd',
                        options=[{'label': b, 'value': b} for b in sorted(df['breed'].dropna().unique())],
                        multi=True,
                        placeholder="Select one or more breeds"
                    ),
                ]),

                html.Div(style={'minWidth': '280px'}, children=[
                    html.Label("Sex Upon Outcome"),
                    dcc.Dropdown(
                        id='sex-dd',
                        options=[{'label': s, 'value': s} for s in sorted(df['sex_upon_outcome'].dropna().unique())],
                        multi=True,
                        placeholder="Select sex (optional)"
                    ),
                ]),

                html.Div(style={'minWidth': '320px'}, children=[
                    html.Label("Age in Weeks (at outcome)"),
                    dcc.RangeSlider(
                        id='age-slider',
                        min=float(np.nanmin(df['age_upon_outcome_in_weeks'])) if 'age_upon_outcome_in_weeks' in df.columns else 0,
                        max=float(np.nanmax(df['age_upon_outcome_in_weeks'])) if 'age_upon_outcome_in_weeks' in df.columns else 200,
                        value=[
                            float(np.nanmin(df['age_upon_outcome_in_weeks'])) if 'age_upon_outcome_in_weeks' in df.columns else 0,
                            float(np.nanmax(df['age_upon_outcome_in_weeks'])) if 'age_upon_outcome_in_weeks' in df.columns else 200,
                        ],
                        step=1,
                        allowCross=False,
                        tooltip={'placement': 'bottom', 'always_visible': True}
                    ),
                    html.Div(id='age-slider-readout', style={'fontSize': '12px', 'color': '#666', 'marginTop': '6px'})
                ]),
            ]
        ),

        html.Br(),
        html.Div(id='filter-summary', style={'fontStyle': 'italic', 'color': '#444'}),
    ]), # <-- closes the whole filter panel cleanly
        
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         
    # Client-friendly features
        page_action='native',
        page_size=10,                 # OK to set; DO NOT persist this
        filter_action='native',
        sort_action='native',
        sort_mode='multi',
        row_selectable='single',      # REQUIRED for the map
        selected_rows=[0],            # pre-select first row
        column_selectable='multi',
        editable=False,

        style_table={'height': '400px', 'overflowY': 'auto', 'overflowX': 'auto'},
        style_cell={
            'minWidth': '120px', 'width': '120px', 'maxWidth': '280px',
            'whiteSpace': 'normal', 'textAlign': 'left', 'fontSize': '12px'
        },
        style_header={'backgroundColor': '#f0f0f0', 'fontWeight': 'bold'},

    # Persistence: ONLY allowed props
        persistence=True,
        persisted_props=['page_current', 'sort_by', 'filter_query', 'selected_rows'],
        persistence_type='session'
    ),
    
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display' : 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Build a server-side MongoDB query for each “rescue type”
def build_rescue_query(rescue_type: str):
    # NOTE: Adjust breeds/age windows to exactly match your Dashboard Specifications Document.
    if rescue_type == 'WATER':
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesa Bay Retr Mix", "Newfoundland Mix"]},
            "sex_upon_outcome": {"$in": ["Neutered Male", "Spayed Female"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'MOUNTAIN':
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": {"$in": ["Neutered Male", "Spayed Female"]},
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }
    elif rescue_type == 'DISASTER':
        return {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinsch", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": {"$in": ["Neutered Male", "Spayed Female"]},
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }
    # Default: ALL (no filter)
    return {}
    
## FIX ME Add code to filter interactive data table with MongoDB queries
#
# Update DataTable when any filter changes
@app.callback(
    Output('datatable-id', 'data'),
    Output('filter-summary', 'children'),
    Output('age-slider-readout', 'children'),
    Input('filter-type', 'value'),
    Input('breed-dd', 'value'),
    Input('sex-dd', 'value'),
    Input('age-slider', 'value'),
)

def update_dashboard(filter_type, breeds, sexes, age_range):
#
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#
    # Build the MongoDB query from the chosen rescue type
    query = build_rescue_query(filter_type)
    
    # Optional breed filter
    if breeds:
        # Intersect with any existing breed $in from rescue query
        if 'breed' in query and '$in' in query['breed']:
            query['breed']['$in'] = list(set(query['breed']['$in']) & set(breeds))
        else:
            query['breed'] = {'$in': breeds}

    # Optional sex filter
    if sexes:
        if 'sex_upon_outcome' in query and '$in' in query['sex_upon_outcome']:
            query['sex_upon_outcome']['$in'] = list(set(query['sex_upon_outcome']['$in']) & set(sexes))
        else:
            query['sex_upon_outcome'] = {'$in': sexes}

    # Age range filter
    if age_range and len(age_range) == 2:
        amin, amax = float(age_range[0]), float(age_range[1])
        if 'age_upon_outcome_in_weeks' in query:
            query['age_upon_outcome_in_weeks'] = {
                '$gte': max(amin, query['age_upon_outcome_in_weeks'].get('$gte', amin)),
                '$lte': min(amax, query['age_upon_outcome_in_weeks'].get('$lte', amax))
            }
        else:
            query['age_upon_outcome_in_weeks'] = {'$gte': amin, '$lte': amax}

    # Fetch from MongoDB using your CRUD module
    results = db.read(query)  # must return a list[dict]
    dff = pd.DataFrame.from_records(results)

    # Drop _id if present (ObjectId breaks DataTable)
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # Ensure types for map if those columns exist (won’t hurt otherwise)
    for col in ('location_lat', 'location_long'):
        if col in dff.columns:
            dff[col] = pd.to_numeric(dff[col], errors='coerce')

    summary = f"Active filter: {filter_type} | Breeds: {len(breeds) if breeds else 0} | Sexes: {len(sexes) if sexes else 0} | Age weeks: {int(age_range[0])}–{int(age_range[1])} | Returned: {len(dff)}"
    readout = f"Selected age range: {int(age_range[0])}–{int(age_range[1])} weeks"
    # Return ONLY the data dict for the DataTable (you set columns at layout time)
    return dff.to_dict('records'), summary, readout
#
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart)

    dff = pd.DataFrame(viewData) if viewData is not None else df.copy()
    if dff.empty or 'breed' not in dff.columns:
        return [html.Div("No data to chart.", style={'color': '#888'})]
    counts = dff['breed'].fillna('Unknown').value_counts()
    top = counts.head(10)
    other = counts.iloc[10:].sum()
    pie_df = top.reset_index()
    pie_df.columns = ['breed', 'count']
    if other > 0:
        pie_df.loc[len(pie_df)] = ['Other', other]
    fig = px.pie(pie_df, names='breed', values='count', title='Animals by Breed (top 10)')
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://candlemonday-channelsong-3000.codio.io/proxy/8050/
